$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $

Here, we update the implementation of the algorithm "Detecting a marked vertex" from the paper Quantum walk speedup of backtracking algorithms by Ashley Montanaro.

We do not use Phase estimation here. Instead, we do a probabilistic sampling:
$$\frac{1}{m} \sum_{i=0}^{m-1} |\bra{00 ... 0} (R_B R_A)^i \ket{00 ... 0}|^2$$

The idea is that we apply the operator $R_B R_A$ to the initial state $\ket{00 ... 0}$ for random number of times (between 0 and $2^k - 1$, where we pick $k$ as the number of bits of precision. If there are no marked vertices, then probability of measuring state $\ket{00 ... 0}$ will not exceed $\frac{1}{2}$, while in case of at least one marked vertex probability will be at least $60 \%$.

Keep in mind that in order to ensure the probabilities we changed transformation $D_r$, where $r$ is denoting root vertex (specifically, state $\psi_r$).

<h2>Parameters</h2>

In [ ]:
num_of_layers = 4
bits_of_precision = 4

<h2>Generating layers - qubit basis state labels</h2>

In [ ]:
layers = [
    ['000'],
    ['100','001'],
    ['110','101','011','111']
]

def add_layer(layer_count,layers):
    new_layers = []
    root = '1'
    for i in range(layer_count-2):
        root = root + '0'
    root = root + '10'
    new_layers.append([root])
    for i in range(len(layers)):
        new_layers.append([])
        for j in range(2**i):
            new_layers[i+1].append('0'+layers[i][j])
        for j in range(2**i):
            new_layers[i+1].append('1'+layers[i][j])
    return new_layers

for layer_count in range(3,num_of_layers+1):
    layers = add_layer(layer_count,layers)

<h3>Verifying correctness of the tree structure</h3>

In [ ]:
correct_bit_count = True
for layer in layers:
    for node in layer:
        if(len(node) != num_of_layers + 1):
            correct_bit_count = False
print('Correct bit count in each node:',correct_bit_count)

no_duplicates = True
for layer in layers:
    for node in layer:
        count = 0
        for layer1 in layers:
            for node1 in layer1:
                if(node == node1):
                    count = count+1
        if(count != 1):
            no_duplicates = False
print('No duplicates:',no_duplicates)

correct_links = True
for i in range(num_of_layers):
    for j in range(2**i):
        vertices = [layers[i][j],layers[i+1][2*j],layers[i+1][2*j+1]]
        same_bits = []
        different_bits = []
        for k in range(num_of_layers+1):
            if(vertices[0][k] == vertices[1][k] and vertices[1][k] == vertices[2][k]):
                same_bits.append(k)
            else:
                different_bits.append(k)
        if(len(different_bits) != 2):
            correct_links = False
print('Correct links:',correct_links)

print('Root:',layers[0][0],'to',layers[1][0],'and',layers[1][1])

<h2>Array for $\psi$</h2>

In this part, we prepare states $\ket{\psi_x}$ for each vertex $x$.

In [ ]:
psi = {}

<h2>Preparing root</h2>

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    0, # 00
    sqrt(num_of_layers*4)/sqrt(1+2*(num_of_layers*4)), # 01
    1/sqrt(1+2*(num_of_layers*4)), # 10
    sqrt(num_of_layers*4)/sqrt(1+2*(num_of_layers*4)) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
# you need to update only 3 operators
psi[layers[0][0]] = QuantumCircuit(num_of_layers+1)
psi[layers[0][0]].x(1)
psi[layers[0][0]].u3(2.9,0,0,num_of_layers-1) # only first parameter needs to be updated
psi[layers[0][0]].u3(1.6,0,0,num_of_layers) # only first parameter needs to be updated
psi[layers[0][0]].cx(num_of_layers,num_of_layers-1)
psi[layers[0][0]].u3(0.245,0,0,num_of_layers-1) # only first parameter needs to be updated
psi[layers[0][0]].cx(num_of_layers,num_of_layers-1)

psi[layers[0][0]].draw(output='mpl')

<h3>Generating parent-children cases</h3>

In [ ]:
cases = {}

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    0, # 00
    1/sqrt(3), # 01
    1/sqrt(3), # 10
    1/sqrt(3) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['00'] = QuantumCircuit(2)
cases['00'].u3(3*pi/4,0,0,0)
cases['00'].u3(1.91,0,0,1)
cases['00'].cx(1,0)
cases['00'].u3(pi/4,0,0,0)
cases['00'].cx(1,0)

cases['00'].draw(output='mpl')

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    1/sqrt(3), # 00
    0, # 01
    1/sqrt(3), # 10
    1/sqrt(3) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['01'] = QuantumCircuit(2)
cases['01'].u3(pi/4,0,0,0)
cases['01'].u3(1.91,0,0,1)
cases['01'].cx(1,0)
cases['01'].u3(-pi/4,0,0,0)
cases['01'].cx(1,0)

cases['01'].draw(output='mpl')

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    1/sqrt(3), # 00
    1/sqrt(3), # 01
    0, # 10
    1/sqrt(3) #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['10'] = QuantumCircuit(2)
cases['10'].u3(3*pi/4,0,0,0)
cases['10'].u3(1.23,0,0,1)
cases['10'].cx(1,0)
cases['10'].u3(-pi/4,0,0,0)
cases['10'].cx(1,0)

cases['10'].draw(output='mpl')

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(3)

desired_vector = [
    1/sqrt(3), # 00
    1/sqrt(3), # 01
    1/sqrt(3), # 10
    0 #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

qc2.draw(output='mpl')

In [ ]:
cases['11'] = QuantumCircuit(2)
cases['11'].u3(pi/4,0,0,0)
cases['11'].u3(1.23,0,0,1)
cases['11'].cx(1,0)
cases['11'].u3(pi/4,0,0,0)
cases['11'].cx(1,0)

cases['11'].draw(output='mpl')

<h3>Generating parent-children circuits</h3>

In [ ]:
series_of_vertex_links = []
for i in range(1,num_of_layers):
    for j in range(2**i):
        vertex_link = [layers[i][j],layers[i+1][2*j],layers[i+1][2*j+1]]
        series_of_vertex_links.append(vertex_link)

In [ ]:
for vertices in series_of_vertex_links:   
    same_bits = []
    different_bits = []
    for i in range(num_of_layers+1):
        if(vertices[0][i] == vertices[1][i] and vertices[1][i] == vertices[2][i]):
            same_bits.append(i)
        else:
            different_bits.append(i)

    current_case = ['00','01','10','11']
    for i in range(3):
        string = '' + vertices[i][different_bits[0]] + vertices[i][different_bits[1]]
        current_case.remove(string)

    psi[vertices[0]] = QuantumCircuit(num_of_layers+1)
    for i in same_bits:
        if(vertices[0][i] == '1'):
            psi[vertices[0]].x(num_of_layers-i)
    psi[vertices[0]] = psi[vertices[0]].compose(cases[current_case[0]],[num_of_layers-different_bits[1],num_of_layers-different_bits[0]])

<h3>Leaves</h3>

In [ ]:
for leaf in layers[num_of_layers]:
    psi[leaf] = QuantumCircuit(num_of_layers+1)
    reversed_index = leaf[::-1]
    for i in range(num_of_layers+1):
        if(reversed_index[i] == '1'):
            psi[leaf].x(i)

<h2>Operators</h2>

In this part, we prepare operators $D_x$ for each vertex $x$. Here $D_x = I - 2 \ket{\psi_x} \bra{\psi_x}$

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

identity_minus_state_zero = QuantumCircuit(num_of_layers+1)

for i in range(num_of_layers+1):
    identity_minus_state_zero.x(i)

control_states = []
for i in range(num_of_layers+1):
    control_states.append(i)


identity_minus_state_zero.h(num_of_layers)
identity_minus_state_zero.mct([*range(num_of_layers)], num_of_layers)
identity_minus_state_zero.h(num_of_layers)

for i in range(num_of_layers+1):
    identity_minus_state_zero.x(i)

d={}
for layer in layers:
    for i in layer:
        d[i] = QuantumCircuit(num_of_layers+1)
        d[i] = d[i].compose(psi[i].inverse(),control_states)
        d[i] = d[i].compose(identity_minus_state_zero)
        d[i] = d[i].compose(psi[i],control_states)

<h2>Implementation</h2>

Here, we repeat experiment with applying $R_BR_A$ for $i$ times for each $0 \leq i \leq 2^k-1$ and gather the total probability of observing state $\ket{00 ... 0}$.

In [ ]:
marked=['11111']

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi
from qiskit.circuit.library import QFT

r_a = QuantumCircuit(num_of_layers+1)

even_distance_layer = []
for i in range(0,num_of_layers+1,2):
    even_distance_layer = even_distance_layer + layers[i]

for i in even_distance_layer:
    if not i in marked:
        r_a = r_a.compose(d[i])
    
r_b = QuantumCircuit(num_of_layers+1)

odd_distance_layer = []
for i in range(1,num_of_layers+1,2):
    odd_distance_layer = odd_distance_layer + layers[i]

for i in odd_distance_layer:
    if not i in marked:
        r_b = r_b.compose(d[i])

rbra = r_a
rbra = rbra.compose(r_b)

print('marked: ['+', '.join(marked)+'], outcome: ')

composed_rbra = QuantumCircuit(num_of_layers+1)
summation = 0
total_counts = {}
for experiment in range(2**bits_of_precision):
    circuit = QuantumCircuit(num_of_layers+1,num_of_layers+1)
    reversed_index = layers[0][0][::-1]
    for i in range(num_of_layers+1):
        if(reversed_index[i] == '1'):
            circuit.x(i)
    circuit = circuit.compose(composed_rbra)
    circuit.measure(range(num_of_layers+1),range(num_of_layers+1))
    job = execute(circuit,Aer.get_backend('qasm_simulator'),shots=10000)
    counts = job.result().get_counts(circuit)
    total_counts_new = {k: total_counts.get(k, 0) + counts.get(k, 0) for k in set(total_counts) | set(counts)}
    total_counts = total_counts_new
    result = counts.get(layers[0][0], 0)
    print('experiment:',experiment)
    print(result/100,'%')
    summation += result
    composed_rbra = composed_rbra.compose(rbra)
print('final probability:',summation/(10000*(2**bits_of_precision)))
ordered_counts = dict(sorted(total_counts.items(), key=lambda item: item[1],reverse=True))
print(ordered_counts)